In [1]:
import numpy as np
from scipy.stats import bernoulli ### (0,1)로 binary encoding된 상황을 만들기 위해서 패키지를 사용

## Model implementation.

In [2]:
import torch.nn as nn
import torch
class RETAIN(nn.Module):
    def __init__(self,input_dim,embedding_dim, hidden_dim) -> None:
        super().__init__()
        
        ##1. 기본적인 parameter 선언
        self.hidden_dim=hidden_dim ## diemension of hidden state, i.e) 128,256, ... 
        self.input_dim=input_dim ## dimension of input time series. 기간을 얼마나 잡을 것인지에 대한 정보를 담음. 
        self.embedding_dim=embedding_dim 
        ## embedded dimension. 단순히 binary encoding의 경우 정보를 많이 저장할 수 없을 수 잇으므로, 정보를 조금더 다채롭게 표현하기 위해서(변수들간의 관계를 고려하기 위해) 
        ##embedding dim 정의 후  embedding 사용.


        ## 2. 임베딩
        self.embedding=nn.Linear(input_dim,embedding_dim)  ## 'Linearly' embedding imput dimension.
        

        ## 3. 양방향으로 RNN 기반의 모델 부르기 
        self.GRUalpha=nn.GRU(embedding_dim,hidden_dim,batch_first=True) ## GRU, for obtaining alphas
        self.GRUbeta=nn.GRU(embedding_dim,hidden_dim,batch_first=True) ## GRU, for obtaining, Betas. 
        
        ## 4. 구해진 시점에서의 timestep 기반으로 alpha, beta구하기 
        self.LinearAlpha=nn.Linear(hidden_dim,1,bias=True) ## Linear Alpha, making hidden state alpha for every timestep (timestep* hiddendim) into (timestep * 1)
        self.LinearBeta=nn.Linear(hidden_dim,embedding_dim,bias=True) ## Linear Beta, making hidden state beta for every timestep( timestep*hiddendim) into (timestep *featurenumber)
        
        
        ## 5. softmax와 tanh를 통해서  각시점에서의 attention value를 구하기 위한 장치 마련. 
        self.Alphsoftmax=nn.Softmax(dim=0) ##(size of timestep)
        self.Betatanh=nn.Tanh() ##(size of timestep)
        


        ## 6. 마지막 Linear layer을 통과시켜, 1(긍정)의 확률 과 0(부정)의 확률을 구해줌. 
        self.lastLinear=nn.Linear(embedding_dim,2) ##  마지막에 결과가 긍정(1)일 확률과 0 일 확률을 구해줘야하므로, 2개의 dimension을 출력하도록 하였음. 
        self.lastSoftmax=nn.Softmax()  

    def forward(self,x):
        h0 = torch.zeros(1, self.hidden_dim)# Initialize hidden state with zeros
        #c0 = torch.zeros(1, x.shape[0], self.hidden_dim) for LSTM, in GRU there is no cell state. 
        
        x=self.embedding(x) ## input 임베딩 


        outalpha, _ = self.GRUalpha(x, (h0))  ## outputsize= timestep* hiddensize

        ## 각각의 timestep에 대하여, 그에 대한 hidden state출력해줌. (1번째 시점에서의 hidden state, 2번째 시점에서의 hidden state... 50번째 시점에서의 hidden state,,) 
        ## 논문에서는 alpha 는 조금더 visit자체에 focus를 둔 attention vector



        outbeta,_=self.GRUbeta(x,(h0)) ## outputsize= timestep* hiddensize

        ## 각각의 timestep에 대하여, 그에 대한 hidden state출력해줌. (1번째 시점에서의 hidden state, 2번째 시점에서의 hidden state... 50번째 시점에서의 hidden state,,) 
        ## 논문에서는 beta는 병들(fetaure)들에 초점을 맞춘 attention vector. 


        attentionalpha=self.LinearAlpha(outalpha) ##timestep *hiddensize-> tiemstep* 1
        attentionbeta=self.LinearBeta(outbeta) ##timestep *hiddensize-> tiemstep* embedding_dim 
        


        attentionalpha=self.Alphsoftmax(attentionalpha)  ##dimension:softmax통과시키는 이유? 가중치를 정규화(normalize에서 )해서 표현하기 우해
        attentionbeta=self.Betatanh(attentionbeta) ##dimension : tanh를 사용하는 이유?  softmax와는 다른 어떤 의미를 capture하기 위해서. 

        

        
        alphabeta=torch.mul(attentionalpha[:,:],attentionbeta[:,:]) ## dimension: timestep. 
        totalsum=torch.sum(alphabeta*x,dim=0) ## sum up attention value to get the probability ,.

        #print(totalsum.shape)
        a=self.lastLinear(totalsum)
        
        #print(a.shape)
        b=self.lastSoftmax(a)
        
        return attentionalpha,attentionbeta,b

## Hyperparameter 정의 

In [4]:
num_feature=10  ## 실제논문에서는 진단명, 처방약 이름 등등을 feature로 사용, 우리문제에 적용한다면, 기업의 사회적/환경적/경영적 등등의 요소들을  지표로 사용해볼 수있음
## 예를 들어, 기업이 12월의 시점에서 좋은 사회공헌을 했다면,  그시점에서 사회공헌에 대한 encoding은 1로사용. 

r = bernoulli.rvs(0.5, size=num_feature) ## feature의 개수만큼(feature을 activate 햇는지 안했는지 여부를 표현하기 위해서)
time_interval=50  ## 얼마정도의 기간을 가지고 예측할건지에 대한 정보. 
hidden_dim=128 ## LSTM/GRU 모델의 hidden layer 개수 256,512, 논문에서는 약 512개 
embedding_dim=200 ## feature관의 관계등을 더 잘 고려하기 위해서, Embedding을 해줘야하는데(자연어처리에서 하는 embedding이랑 비슷) 어느정도로 고려해줘야 하는지에 대한 정보. 

### 실제 retain모델이나 저희가 구현할때도 timestep을 reversed order로 바꿔주고 진행해야하지만, 예시로 보여드리는거기때문에 그 과정은 없이 진행했습니다.
- reversed order로 진행하는 이유?=> rnn계열의 모델에 각각의 timestep마다 hiddenstate를 출력하는데 순차적으로 넣게 된다면, 첫번째 timestep은 독립적인(다른 timestep과 관련없는 hiddenstate를 출력하기 때문에 조금더
다채로운 해석을 위해서 `)

In [5]:
timeinfo=[] #임의로 time -feature variable 생성, 각시간간격마다 약 200개의 feature 존재. 
for i in range(time_interval):
    r = bernoulli.rvs(0.5, size=num_feature)
    timeinfo.append(r)

In [6]:
model=RETAIN(num_feature,embedding_dim,hidden_dim)

In [7]:
timeinfo=np.array(timeinfo) 

In [8]:
a=torch.from_numpy(timeinfo).float()

In [9]:
a.shape

torch.Size([50, 10])

In [10]:
alpha,beta,logit=model(a)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_25152\1987887076.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  b=self.lastSoftmax(a)


## Coefficient for each data $x_{ij}$

In [11]:
alpha.shape ## attention weight for each timestep

torch.Size([50, 1])

In [12]:
beta.shape ##attention weight for each time step and value. 

torch.Size([50, 200])

### 나오는 output은 각각의  timestep 마다, feature의 weight을 구해주는 output. 
- 각각의 feature 에대하여 총 50개의 timestep이 존재하므로, 500개의 결과(timestep*inputvalue)
- 결과가  size가 2인 1d  tensor 형태로 나오는데, 첫번째 값은 결과가 부정(0)일때  해당 input value가 기여한 정도, 두번째 값은 결과가 긍정(1)일 때 해당 input value 가 기여한 정도 

In [13]:
timeinfo

array([[1, 1, 0, 0, 1, 1, 1, 0, 1, 0],
       [1, 0, 0, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 1, 0, 1, 1],
       [1, 1, 0, 1, 1, 1, 1, 0, 0, 1],
       [1, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 1, 1, 0, 1, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
       [0, 1, 1, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [1, 1, 0, 1, 0, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
       [1, 1, 1, 1, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 1, 0, 1, 0, 1],
       [1, 1, 0, 0, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 1, 1, 1, 1,

#### 결과가 전부다 0 근처에서 나오는데, 따로 backpropagation을 진행한 것이 아니라, forward propagation만 진행했기 때문에 그렇습니다. 

In [14]:

for i in range(time_interval):
    alpha[i,:]
    valueimpact=(beta[i,:])
    for k in range(num_feature):
        if timeinfo[i,k]==0:
            continue

        weight=(model.embedding.weight[:,k])
        weight*valueimpact
        ans=model.lastLinear(weight*valueimpact)*alpha[i,:]
        print("effect of timestep {} and feature {} for obtaining Positive Result:  {}".format(i+1,k+1,ans[1]))




effect of timestep 1 and feature 1 for obtaining Positive Result:  0.000930696667637676
effect of timestep 1 and feature 2 for obtaining Positive Result:  0.0011387962149456143
effect of timestep 1 and feature 5 for obtaining Positive Result:  0.0008513569482602179
effect of timestep 1 and feature 6 for obtaining Positive Result:  0.0011667622020468116
effect of timestep 1 and feature 7 for obtaining Positive Result:  0.0013367502251639962
effect of timestep 1 and feature 9 for obtaining Positive Result:  0.0013911302667111158
effect of timestep 2 and feature 1 for obtaining Positive Result:  0.0012180424528196454
effect of timestep 2 and feature 4 for obtaining Positive Result:  0.0017243364127352834
effect of timestep 2 and feature 6 for obtaining Positive Result:  0.0013485541567206383
effect of timestep 2 and feature 7 for obtaining Positive Result:  0.0010995280463248491
effect of timestep 2 and feature 8 for obtaining Positive Result:  0.0012370164040476084
effect of timestep 3 a